In [ ]:
# Generate Spark Session
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("Reading from Sockets")
    .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", True)
    .master("spark://spark-master:7077")
    .getOrCreate()
)

spark

In [ ]:
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", True)

# AQE DOES NOT WORK FOR STREAMING APPLICATIONS
spark.conf.set("spark.sql.adaptive.enabled", False)
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", False)

# SEE DIFFERENCE IN PROCESSING TIME BY CHANGING THIS PARAMETER
spark.conf.set("spark.sql.shuffle.partitions", 1)

In [ ]:
# IN A TERMINAL OF JUPYTER
# ncat -l 9999
# write lines of text + ENTER so spark receives them through the socket connection

In [ ]:
# Read input data

df_raw = (
    spark.readStream.format("socket")
    .option("host", "localhost")
    .option("port", "9999")
    .load()
)

In [ ]:
df_raw.printSchema()

In [ ]:
# Split the line into words
from pyspark.sql import functions as F


def process_df_words(df_raw):
    df_words = df_raw.withColumn("words", F.split("value", " "))
    df_explode = df_words.withColumn("word", F.explode("words")).drop("value", "words")
    df_agg = df_explode.groupBy("word").count()
    return df_agg

In [ ]:
df_agg = process_df_words(df_raw)

In [ ]:
df_agg.printSchema()

In [ ]:
# Write the output to console streaming

df_agg.writeStream.format("console").outputMode("complete").start().awaitTermination()

In [12]:
spark.stop()